In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from prophet import Prophet
import warnings
from math import ceil
from tqdm.auto import tqdm
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_absolute_percentage_error, root_mean_squared_error
import json
import random
import logging


from datetime import datetime, timedelta

Importing plotly failed. Interactive plots will not work.


In [2]:
logging.getLogger("prophet").setLevel(logging.ERROR)
logging.getLogger("cmdstanpy").setLevel(logging.ERROR)

In [3]:
# Load the dataset
data = pd.read_csv('ts_hist.csv')

In [4]:
data['date'] = pd.to_datetime(data['date']) 

In [5]:
prod_ids=data['index'].drop_duplicates().tolist()
prod_ids.sort()

In [6]:
start_date = datetime.strptime('01012014', "%d%m%Y").date()
end_date = datetime.strptime('15052016', "%d%m%Y").date()

td=(end_date-start_date).days
#print(td)

date_list = [start_date + timedelta(days=x) for x in range(td+1) if (start_date + timedelta(days=x)).weekday()==6 ]

# for p_id in tqdm(prod_ids):

#     for d in tqdm(date_list):
#         if d not in data[data['index']==p_id]['date'].values:
#             data2 = pd.DataFrame({'date':[d],'qnt':[0],'index':p_id})
#             #w_data = w_data.append({'ds':[d],'y':[0]},ignore_index=True)
#             data = pd.concat([data, data2], ignore_index = True)

In [7]:
data['Year_Number'] = data['date'].dt.isocalendar().year
data['Week_Number'] = data['date'].dt.isocalendar().week
data["Weekly_qnt"]=data.groupby(
    ["Year_Number",'Week_Number',"index"])["qnt"].transform('sum')

In [8]:
data["Weekly_date"]=data.groupby(
    ["Year_Number",'Week_Number'])["date"].transform('max')

In [9]:
# df_holidays = data.loc[~data['event_name_1'].isnull()][['Weekly_date','event_name_1']].sort_values(by=['Weekly_date'])
# df_holidays=df_holidays.rename(columns={"Weekly_date": "ds", "event_name_1": "holiday"}, errors="raise").drop_duplicates().reset_index()
# df_holidays=df_holidays.drop(['index'], axis=1)

df_holidays = data.loc[~data['event_type_1'].isnull()][['Weekly_date','event_type_1']].sort_values(by=['Weekly_date'])
#df_holidays = df_holidays.loc[df_holidays['event_type_1'].isin(['Sporting','National','Religious'])] # Cultural National Religious Sporting
df_holidays=df_holidays.rename(columns={"Weekly_date": "ds", "event_type_1": "holiday"}, errors="raise").drop_duplicates().reset_index()
df_holidays=df_holidays.drop(['index'], axis=1)

In [10]:
df_data = data[['Weekly_date','Weekly_qnt','index']].sort_values(by=['Weekly_date'])
df_data=df_data.rename(columns={"Weekly_date": "ds", "Weekly_qnt": "y"}, errors="raise").drop_duplicates().reset_index()
df_data=df_data.drop(['level_0'], axis=1)

In [37]:
#complete_idx = pd.MultiIndex.from_product((set(df_data.ds),set(df_data['index'])))

In [39]:
#all_data = df_data.set_index(['ds','index']).reindex(complete_idx, fill_value=0).reset_index()

In [ ]:
#df_data=all_data.sort_values(by=['ds'])

In [11]:
# params_grid = {'seasonality_mode':('multiplicative','additive'),
#                'changepoint_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'holidays_prior_scale':[0.1,0.2,0.3,0.4,0.5],
#               'n_changepoints' : [100,150,200],
#               }

params_grid = dict(growth = ['linear'], # 'logistic', 'flat'
                   n_changepoints  = range(30, 100, 10),
                   changepoint_range  = [0.1, 0.2, 0.3, 0.4, 0.5,],
                   yearly_seasonality = [True],
                   #weekly_seasonality = [True, False],
                   #daily_seasonality = [True, False],
                   seasonality_mode = ['multiplicative'],
                   seasonality_prior_scale=range(1, 5),
                   changepoint_prior_scale=[0.01],
                   interval_width = [0.3, 0.4, 0.5],
                   uncertainty_samples = [300],
                   holidays_prior_scale = [0.1, 0.2, 0.3]
                   )

grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 1260


In [12]:
result=[]

In [26]:
for prod_id in tqdm(prod_ids):
    w_data=df_data.loc[df_data['index'] == prod_id].sort_values(by=['ds'])
    w_data=w_data.drop(['index'], axis=1)
    dates=w_data['ds'].tolist()
    dates_diff = [x for x in date_list if pd.Timestamp(x) not in dates]
    
    dates_ds=[{'ds':pd.Timestamp(x), 'y':0} for x in dates_diff]
    dates_df = pd.DataFrame(dates_ds)
    dates_df
    w_data = pd.concat([w_data, dates_df], ignore_index = True)
    w_data=w_data.sort_values(by=['ds'])
    df_weekly = w_data.iloc[:-10]
    w_test = w_data.iloc[-10:]

    
    
    model_parameters = pd.DataFrame(columns = ['MAPE','Parameters'])
    random.seed(25)
    for i in tqdm(range(100)):
        
        p=grid[random.randint(0, cnt)]
        test = pd.DataFrame()
        #print(p)
        
        train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                             holidays_prior_scale = p['holidays_prior_scale'],
                             n_changepoints = p['n_changepoints'],
                             changepoint_range = p['changepoint_range'],
                             seasonality_mode = p['seasonality_mode'],
                             growth=p['growth'],
                             #weekly_seasonality=True,
                             #daily_seasonality = True,
                             yearly_seasonality = p['yearly_seasonality'],
                             seasonality_prior_scale = p['seasonality_prior_scale'],                         
                             holidays=df_holidays, 
                             uncertainty_samples = p['uncertainty_samples'],
                             interval_width=p['interval_width'])
        train_model.add_country_holidays(country_name='US')
        train_model.add_seasonality(name='yearly', period=365, fourier_order=5)
        train_model.fit(df_weekly)
        train_forecast = train_model.make_future_dataframe(periods=10, freq='W',include_history = False)
        train_forecast = train_model.predict(train_forecast)
        test=train_forecast[['ds','yhat']]
        Actual = w_test
        
        MAPE = mean_absolute_percentage_error(Actual['y'],abs(test['yhat']))
        #MAPE = root_mean_squared_error(Actual['y'],test['yhat'])
        print('Mean Absolute Percentage Error(MAPE)------------------------------------',MAPE)    
        #model_parameters = model_parameters.append({'MAPE':MAPE,'Parameters':p},ignore_index=True)
        df2 = pd.DataFrame({'MAPE':[MAPE],'Parameters':[p]})
        model_parameters = pd.concat([model_parameters, df2], ignore_index = True)
    
    parameters = model_parameters.sort_values(by=['MAPE'])
    parameters = parameters.reset_index(drop=True)
    p=parameters['Parameters'][0]
    
    
    
    
    mp = Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                             holidays_prior_scale = p['holidays_prior_scale'],
                             n_changepoints = p['n_changepoints'],
                             changepoint_range = p['changepoint_range'],
                             seasonality_mode = p['seasonality_mode'],
                             growth=p['growth'],
                             #weekly_seasonality=True,
                             #daily_seasonality = True,
                             yearly_seasonality = p['yearly_seasonality'],
                             seasonality_prior_scale = p['seasonality_prior_scale'],                         
                             holidays=df_holidays, 
                             uncertainty_samples = p['uncertainty_samples'],
                             interval_width=p['interval_width'])
    mp.add_country_holidays(country_name='US')
    mp.add_seasonality(name='yearly', period=365, fourier_order=5)
    mp.fit(w_data)
    future = mp.make_future_dataframe(freq='W', periods=1)
    forecast = mp.predict(future)
    res=ceil(forecast.iloc[-1][['yhat']])
    result.append({'index':prod_id,'y':res})
    

  0%|          | 0/405 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

21:28:03 - cmdstanpy - INFO - Chain [1] start processing
21:28:03 - cmdstanpy - INFO - Chain [1] done processing
21:28:03 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
21:28:03 - cmdstanpy - INFO - Chain [1] start processing
21:28:03 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\yaros\AppData\Local\Temp\ipykernel_17648\2350604122.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  model_parameters = pd.concat([model_parameters, df2], ignore_index = True)


Mean Absolute Percentage Error(MAPE)------------------------------------ 2.042291197081757e+17


21:28:03 - cmdstanpy - INFO - Chain [1] start processing
21:28:03 - cmdstanpy - INFO - Chain [1] done processing
21:28:04 - cmdstanpy - INFO - Chain [1] start processing


...

22:18:28 - cmdstanpy - INFO - Chain [1] done processing
22:18:28 - cmdstanpy - INFO - Chain [1] start processing


Mean Absolute Percentage Error(MAPE)------------------------------------ 2.3470731280025867


22:18:28 - cmdstanpy - INFO - Chain [1] done processing
22:18:28 - cmdstanpy - INFO - Chain [1] start processing


Mean Absolute Percentage Error(MAPE)------------------------------------ 2.7121661148747025


22:18:28 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\yaros\AppData\Local\Temp\ipykernel_17648\2350604122.py:78: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  res=ceil(forecast.iloc[-1][['yhat']])


In [27]:
 submission = pd.DataFrame(result)

In [28]:
submission.to_csv("submit_f2.csv",index=False)